# Vertex AI Search Agent Builder

## Pre-requisite Setup

### Enable APIs and service account permissions

Enable the Vertex AI Search API:
```
gcloud services enable discoveryengine.googleapis.com
```
Enable the Enterprise Knowledge Graph API:
```
gcloud services enable enterpriseknowledgegraph.googleapis.com
```
Enable Cloud Run:
```
gcloud services enable run.googleapis.com
```
Give the Cloud Run service account required permissions:
```
gcloud projects add-iam-policy-binding [PROJECT_ID or PROJECT_NUMBER] --member='serviceAccount:[PROJECT_NUMBER]-compute@developer.gserviceaccount.com' --role='roles/discoveryengine.viewer'
```

### Install Dependencies and set variables

In [ ]:
! pip3 install -q google-cloud-discoveryengine

In [ ]:
import os
import socket
import re
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]
LOCATION = "global"

UNIQUE_PREFIX = socket.gethostname()
DATASTORE_NAME = re.sub('-', '_', UNIQUE_PREFIX)
DATASTORE_ID = f"{DATASTORE_NAME}_datastore"

## Setting up Agent Builder

### Creating a Datastore

Data store's ingest data for your search app, including scraping websites, bigquery, google drive etc. ([reference](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es))

In [ ]:
def create_data_store(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical="GENERIC",
        content_config="CONTENT_REQUIRED",
    )

    request = discoveryengine.CreateDataStoreRequest(
        parent=discoveryengine.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        data_store=data_store,
        data_store_id=data_store_id,
    )
    operation = client.create_data_store(request=request)

    # Make the request
    # The try block is necessary to prevent execution from halting due to an error being thrown when the datastore takes a while to instantiate
    try:
        response = operation.result(timeout=90)
    except:
        print("long-running operation")

In [ ]:
try:
    create_data_store(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID)
except Exception as e:
    print("Ignore if datastore already exists, if first time, delete existing engine or change name")
    print(f"Error: {e}")

### Import Documents

In [ ]:
def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: str,
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    source_documents = [f"{gcs_uri}/*"]

    request = discoveryengine.ImportDocumentsRequest(
        parent=parent,
        gcs_source=discoveryengine.GcsSource(
            input_uris=source_documents, data_schema="content"
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
    )

    # Make the request
    operation = client.import_documents(request=request)

    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    return operation.operation.name

In [ ]:
source_documents_gs_uri = (
    "gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs"
)
try:
    import_documents(PROJECT_ID, LOCATION, DATASTORE_ID, source_documents_gs_uri)
except Exception as e:
    print(f"Error: {e}")

### Creating a Search Engine

In [ ]:
def create_engine(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.EngineServiceClient(client_options=client_options)

    # Initialize request argument(s)
    config = discoveryengine.Engine.SearchEngineConfig(
        search_tier="SEARCH_TIER_ENTERPRISE", search_add_ons=["SEARCH_ADD_ON_LLM"]
    )

    engine = discoveryengine.Engine(
        display_name=data_store_name,
        solution_type="SOLUTION_TYPE_SEARCH",
        industry_vertical="GENERIC",
        data_store_ids=[data_store_id],
        search_engine_config=config,
    )

    request = discoveryengine.CreateEngineRequest(
        parent=discoveryengine.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        engine=engine,
        engine_id=engine.display_name,
    )

    # Make the request
    operation = client.create_engine(request=request)
    response = operation.result(timeout=90)

In [ ]:
try:
    create_engine(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID)
except Exception as e:
    print("Ignore if engine already exists, if first time, delete existing engine or change name")
    print(f"Error: {e}")

### Query Datastore

In [ ]:
def search_sample(
    project_id: str,
    location: str,
    data_store_id: str,
    search_query: str,
) -> list[discoveryengine.SearchResponse]:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if LOCATION != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        serving_config="default_config",
    )

    # Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response

In [ ]:
query = "Who is the CEO of Google?"

print(search_sample(PROJECT_ID, LOCATION, DATASTORE_ID, query).summary.summary_text)

## Creating Search App

In [ ]:
! git clone https://github.com/GoogleCloudPlatform/generative-ai.git

In [142]:
! pip3 install -q -r generative-ai/search/web-app/requirements.txt

### Deploy Cloud Run function

In [143]:
! gcloud run deploy vertex-ai-search-demo --allow-unauthenticated --region=asia-southeast1 -q --source generative-ai/search/web-app/.

Building using Dockerfile and deploying container to Cloud Run service [vertex-ai-search-demo] in project [ai-sb-test] region [asia-southeast1]
Building and deploying...                                                      
  . Uploading sources...                                                       
  . Building Container...                                                      
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      
  Building and deploying...                                                    





⠛ Building and deploying...                                                    
  ⠛ Uploading sources...                                                       




⠹ Building and deploying... Uploading sources.                                 
  ⠹ Uploading sources...                       